# Wrangling, Analysis & Visualization: @WeRateDogs Tweets

##### Import packages

In [33]:
import pandas as pd
import numpy as np
import requests
import json
import tweepy
import config

# Replace config variables with your own API keys.
consumer_key = config.consumer_key  
consumer_secret = config.consumer_secret  
access_token = config.access_token
access_secret = config.access_secret

auth = tweepy.OAuthHandler(consumer_key, consumer_secret)
auth.set_access_token(access_token, access_secret)

api = tweepy.API(auth)

## Gather

##### WeRateDogs Twitter Archive

In [5]:
twitter_archive_enhanced = pd.read_csv('twitter-archive-enhanced.csv')

##### Neural Network Image Predictions downloaded from Udacity's servers

In [56]:
url = 'https://d17h27t6h515a5.cloudfront.net/topher/2017/August/599fd2ad_image-predictions/image-predictions.tsv'
r = requests.get(url)

open('image-predictions.tsv', 'wb').write(r.content)

image_predictions = pd.read_csv('image-predictions.tsv', sep='\t')
image_predictions.head()

## Assess

## Clean

#### Define

#### Code

#### Test